In [1]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import os
import tensorflow as tf

In [2]:
trainDataset=pd.read_csv("../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train.csv")
trainDataset.head()

,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


In [3]:
inputShape=224

In [4]:
def load_images_from_folder(folder):
    images = {}
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images[filename]=cv2.resize(img,(inputShape,inputShape))
    return images

In [5]:
trainImageData=load_images_from_folder("../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train/")

In [ ]:
(trainImageData["image3476.jpg"])

In [ ]:
plt.figure(figsize=(2,2))
plt.imshow(trainImageData["image3476.jpg"])

In [ ]:
trainImageData["image3476.jpg"].dtype

In [6]:
images=[]
for item in trainDataset["Image"]:
    images.append(trainImageData[item].astype(np.float32))
images=np.array(images)

In [7]:
del trainImageData

In [ ]:
trainDataset["Class"].unique()

In [8]:
labels={"Miscellaneous":0,"Candle":1,"Snowman":2,"Airplane":3,"Christmas_Tree":4,"Jacket":5}
labels_name=list(labels.keys())

In [9]:
trainDataset["Labels"]=trainDataset["Class"].map(lambda x:labels[x])

In [ ]:
trainDataset.head()

In [ ]:
trainDataset["Class"].size

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

In [10]:
CNN=keras.Sequential([

# preprocessing layers
#     keras.layers.experimental.preprocessing.RandomFlip("horizontal",input_shape=(inputShape,inputShape,3)),
#     keras.layers.experimental.preprocessing.RandomRotation(0.2),
#     keras.layers.experimental.preprocessing.RandomZoom(0.2),


    keras.applications.ResNet152(include_top=False,weights="imagenet",input_shape=(inputShape,inputShape,3),classifier_activation="sigmoid"),




# flatten layer
    keras.layers.Flatten(),

# dropout layer
    keras.layers.Dropout(0.2),

# dense layer
    keras.layers.Dense(2560,activation="sigmoid"),
    
# dropout layer
    keras.layers.Dropout(0.2),

#output layer
    keras.layers.Dense(6,activation="softmax")
])

CNN.compile(
    optimizer="sgd",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)



234700800/234698864 [==============================] - 2s 0us/step


In [11]:
with tf.device("/device:GPU:0"):
  CNN.fit(images,trainDataset[["Labels"]],epochs=20,validation_split=0.1)

Epoch 1/20
182/182 [==============================] - 80s 438ms/step - loss: 0.5578 - accuracy: 0.8231 - val_loss: 0.3808 - val_accuracy: 0.8856
Epoch 2/20
182/182 [==============================] - 76s 419ms/step - loss: 0.0880 - accuracy: 0.9737 - val_loss: 0.2329 - val_accuracy: 0.9196
Epoch 3/20
182/182 [==============================] - 76s 419ms/step - loss: 0.0373 - accuracy: 0.9921 - val_loss: 0.2196 - val_accuracy: 0.9258
Epoch 4/20
182/182 [==============================] - 76s 419ms/step - loss: 0.0241 - accuracy: 0.9940 - val_loss: 0.2031 - val_accuracy: 0.9397
Epoch 5/20
182/182 [==============================] - 76s 419ms/step - loss: 0.0194 - accuracy: 0.9957 - val_loss: 0.2035 - val_accuracy: 0.9335
Epoch 6/20
182/182 [==============================] - 76s 420ms/step - loss: 0.0168 - accuracy: 0.9960 - val_loss: 0.2090 - val_accuracy: 0.9304
Epoch 7/20
182/182 [==============================] - 76s 419ms/step - loss: 0.0132 - accuracy: 0.9964 - val_loss: 0.2138 - val_ac

In [12]:
del images

In [13]:
testImageData=load_images_from_folder("../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test")

In [ ]:
plt.figure(figsize=(2,2))
plt.imshow(testImageData["image7.jpg"])

In [14]:
testImages=[]
for i in testImageData:
    testImages.append(testImageData[i].astype(np.float32))
testImages=np.array(testImages)

In [28]:
testImages[0].dtype

dtype('float32')

In [15]:
predicted=CNN.predict(testImages)

In [16]:
predicted_classes=[]
for i in predicted:
    predicted_classes.append(labels_name[np.argmax(i)])

In [17]:
testimagenames=list(testImageData.keys())

In [32]:
testimagenames[0],predicted_classes[0]

('image7761.jpg', 'Airplane')

In [18]:
output=list(zip(testimagenames,predicted_classes))
output_csv=pd.DataFrame(output,columns=["Image","Class"])

In [19]:
output_csv.to_csv("submission.csv",index=None)

In [21]:
CNN.save("model94")